In [1]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [2]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [6]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "gXSGt9fAmDyTyZMSod7NA4peX"
    consumer_secret = "w4mWlV7RPH50WUgUg80FWCOJYb2vmrNIyZac4yzp2AW9Ymclef"
    access_token = "1514341472396476417-5R8bUmn6l4ZkDD2WJFUYVEY19o40CQ"
    access_token_secret = "vt3BtR4mxAaaWKDb1cije5Kwlrw5nV6ooeegbsYxQyXT1"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    # LOCATIONS = [-10.686509, 50.175636, 1.806661, 60.827529]  # UK and Ireland
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472]  # Contiguous US

    stream_listener = StreamListener(time_limit=180, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1516463279027240962, 'username': 'certainlynotal', 'created_at': 'Tue Apr 19 17:06:34 +0000 2022', 'lng': -76.30038250000001, 'lat': 40.0399505, 'text': 'Hope I get avian flu and grow chicken wings'}
{'id': 1516463279362695168, 'username': 'trash_bandic0ot', 'created_at': 'Tue Apr 19 17:06:34 +0000 2022', 'lng': -87.732013, 'lat': 41.8335845, 'text': '"BECOME LEGEND, YOUNG BOY!"'}
{'id': 1516463280583180290, 'username': 'Mediatingmimi', 'created_at': 'Tue Apr 19 17:06:35 +0000 2022', 'lng': -82.36921057, 'lat': 36.303326, 'text': 'Fun evening building with dominos last night and watching the “Lily Topples the World” @hevesh5 you’re an inspirati… https://t.co/uaqDzJjWjP'}
{'id': 1516463283670294530, 'username': 'Short_Queer', 'created_at': 'Tue Apr 19 17:06:36 +0000 2022', 'lng': -71.0887125, 'lat': 42.3136695, 'text': '@spookyy_action your mind…'}
{'id': 1516463287126343688, 'username': 'TheAnimalsKnow', 'created_at': 'Tue Apr 19 17:06:36 +0000 2022', 'lng': -82.56517600000001, 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
